# **Day_03**

## *Part 1/2*

### Instructions

--- Day 3: Crossed Wires ---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?


### Imports

In [1]:
import pathlib

### Inputs

In [2]:
# Define file path for inputs
path = pathlib.Path("inputs/day_03.txt")

# Read and convert to list
diagram_raw = path.read_text().split("\n")

# Split out the wires to individual lists
wire_1 = diagram_raw[0].split(',')
wire_2 = diagram_raw[1].split(',')

### Solution

#### Define the tracer function

In [123]:
# Define our function
def trace_wires(wire):
    global position_history
    # Set position to center
    current_position = [0,0]
    # Save as to find crossed paths later
    position_history = []
    
    # Iterate through each wire segment
    for ix,segment in enumerate(wire):
        print('='*20,f"Index {ix}",f"Segment {segment}",'='*20)

        direction = segment[0]
        distance = int(segment[1:])

        #print(f"Dir:{direction} Dist:{distance}")

        for i in range(distance):
            # If up
            if direction == 'U':
                step = tuple([current_position[0]+1,current_position[1]])

            # If down
            elif direction == 'D':
                step = tuple([current_position[0]-1,current_position[1]])

            # If left
            elif direction == 'L':
                step = tuple([current_position[0],current_position[1]-1])

            # If right
            elif direction == 'R':
                step = tuple([current_position[0],current_position[1]+1])
            
            #print("step:",step)
            #print("current pos:",current_position)
            current_position = step
            position_history.append(current_position)
            
        #position_history.append(tuple(current_position))
        #print(f"Current position is now {current_position}")
        
    return position_history
    


#### Trace the history/paths of each wire

In [124]:
wire_histories = []

for wire in [wire_1,wire_2]:
    wire_history = trace_wires(wire)
    wire_histories.append(wire_history)

==================== Index 0 Segment R995 ====================
==================== Index 1 Segment D882 ====================
==================== Index 2 Segment R144 ====================
==================== Index 3 Segment U180 ====================
==================== Index 4 Segment L638 ====================
==================== Index 5 Segment U282 ====================
==================== Index 6 Segment L907 ====================
==================== Index 7 Segment D326 ====================
==================== Index 8 Segment R731 ====================
==================== Index 9 Segment D117 ====================
==================== Index 10 Segment R323 ====================
==================== Index 11 Segment U529 ====================
==================== Index 12 Segment R330 ====================
==================== Index 13 Segment U252 ====================
==================== Index 14 Segment R73 ====================
==================== Index 15 Segment U173 ========

#### Compare paths for closest to origin

In [125]:
wire_1_history = set(wire_histories[1]) 

shortest_crossed_path = 1e9
for ix,position in enumerate(wire_histories[0]):
    if position == (0,0):
        continue
    if position in wire_1_history:
        distance = abs(position[1] - 0) + abs(position[0] - 0)
        if distance < shortest_crossed_path:
            shortest_crossed_path = distance
            print(f"Found new shortest path: {position}")
            print(f"Distance: {distance}")

Found new shortest path: (1383, 2487)
Distance: 3870
Found new shortest path: (1428, 1381)
Distance: 2809
Found new shortest path: (1159, 970)
Distance: 2129


## *Part 2/2*

### Instructions

### Solution

2+2

In [ ]:
2+2